**Step 1: Get Data Set from Google Drive**

Uploaded to google drive is a folder "Weapons2" containing 5106 .png images and .txt files with the segmentations of weapons.

[This Github Respository](https://github.com/GraceMcPadden2/CSE4830-Final-Project/tree/main/ML%20project) is the code clean_data.py that we wrote to convert each .xaml label with the segmenations to a normalized label in the format used by YOLOv5, YOLOv8, and YOLOv8. This repository also contains a file seperate.py which seperates the images and txt files into seperate directories.

In [ ]:
!nvidia-smi

from google.colab import drive
drive.mount('/content/drive')

!unzip -q /content/drive/MyDrive/Weapons2 -d /content/Weapons2

Tue May  6 06:02:53 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   38C    P8             12W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

**Step 2: Prepare files**

The files were split into train, test, and validation. The split used is 70% train, 15% test, and 15% validation.

A .yaml file is also created containing information about the categories and files for use in YOLO models.

In [ ]:
import os
import random
import shutil

# Config
source_txt = '/content/Weapons2/Weapons/txt'
source_img = '/content/Weapons2/Weapons/jpg'

train_txt = '/content/train/labels'
train_img = '/content/train/images'
val_txt = '/content/val/labels'
val_img = '/content/val/images'
test_txt = '/content/test/labels'
test_img = '/content/test/images'

# Create folders
for folder in [train_txt, train_img, val_txt, val_img, test_txt, test_img]:
    os.makedirs(folder, exist_ok=True)

# Get all txt files and shuffle
all_txt_files = [f for f in os.listdir(source_txt) if f.endswith('.txt')]
random.shuffle(all_txt_files)

# Split ratios
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

n = len(all_txt_files)
train_files = all_txt_files[:int(n * train_ratio)]
val_files = all_txt_files[int(n * train_ratio):int(n * (train_ratio + val_ratio))]
test_files = all_txt_files[int(n * (train_ratio + val_ratio)):]

def move_files(file_list, target_txt_folder, target_img_folder):
    for txt_file in file_list:
        txt_src = os.path.join(source_txt, txt_file)
        jpg_name = os.path.splitext(txt_file)[0] + '.jpg'
        jpg_src = os.path.join(source_img, jpg_name)

        # Copy txt
        if os.path.exists(txt_src):
            shutil.copy(txt_src, os.path.join(target_txt_folder, txt_file))
        else:
            print(f"Missing TXT: {txt_src}")

        # Copy jpg
        if os.path.exists(jpg_src):
            shutil.copy(jpg_src, os.path.join(target_img_folder, jpg_name))
        else:
            print(f"Missing JPG: {jpg_src}")

# Move files
move_files(train_files, train_txt, train_img)
move_files(val_files, val_txt, val_img)
move_files(test_files, test_txt, test_img)

print(f"Done!")
print(f"{len(train_files)} train, {len(val_files)} val, {len(test_files)} test samples")

Done!
3574 train, 766 val, 766 test samples


In [ ]:
yaml_content = """
train: /content/train/images
val: /content/test/images
test: /content/test/images

nc: 3
names: [Handgun, Short_rifle, Knife]
"""

with open('/content/data.yaml', 'w') as f:
    f.write(yaml_content)

**Step 3: Train the models**

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 56.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 108.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

YOLOV5s

In [ ]:
##To tune hyperparameters, create a hyp.yaml file

yaml_content = """
lr0: 0.001

lrf: 0.01
momentum: 0.937
weight_decay: 0.0005
warmup_epochs: 3.0
warmup_momentum: 0.8
warmup_bias_lr: 0.1
box: 0.05
cls: 0.5
cls_pw: 1.0
obj: 1.0
obj_pw: 1.0
iou_t: 0.20
anchor_t: 4.0
fl_gamma: 0.0
hsv_h: 0.015
hsv_s: 0.7
hsv_v: 0.4
degrees: 0.0
translate: 0.1
scale: 0.5
shear: 0.0
perspective: 0.0
flipud: 0.0
fliplr: 0.5
mosaic: 1.0
mixup: 0.0
copy_paste: 0.0
"""

with open('/content/hyp.yaml', 'w') as f:
    f.write(yaml_content)

In [ ]:
# %cd /content/
# !git clone https://github.com/ultralytics/yolov5
# %cd yolov5
# !pip install -r requirements.txt

!python train.py --img 640 --batch 16 --epochs 30 --data /content/data.yaml --weights yolov5s.pt --cache --hyp /content/hyp.yaml
!cp -r /content/runs/train/exp /content/drive/MyDrive/RESULTS_FINAL/

Streaming output truncated to the last 5000 lines.
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100% 24/24 [00:03<00:00,  7.11it/s]
                   all        766        393      0.691      0.324      0.348      0.126

      Epoch    GPU_mem   box_loss   obj_loss   cls_loss  Instances       Size
  0% 0/224 [00:00<?, ?it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      19/29      3.55G    0.04291   0.003528    0.01179         10        640:   0% 1/224 [00:00<00:26,  8.42it/s]/content/yolov5/train.py:412: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(amp):
      19/29      3.55G    0.04124    0.00385   0.009414         19        640:   1% 2/224 [00:00<00:29,  7.63it/s]/content/yolov5/trai

YOLOV8s

In [ ]:
# Install YOLOv8
!pip install ultralytics

from ultralytics import YOLO

model = YOLO('yolov8s.pt')

# Train the model
model.train(
    data='/content/data.yaml',
    imgsz=640,
    batch=16,
    epochs =30,
    cache=True
)

100%|██████████| 21.5M/21.5M [00:00<00:00, 424MB/s]

Ultralytics 8.3.127 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)


engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=cuda:0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=None, rect=False, resume=False, retina_masks=False, save=True,

100%|██████████| 5.35M/5.35M [00:00<00:00, 310MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3779.3±1159.9 MB/s, size: 291.6 KB)


...

                   all        766        387      0.759      0.408      0.486      0.208



30 epochs completed in 0.313 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 22.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.127 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
Model summary (fused): 72 layers, 11,126,745 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 24/24 [00:02<00:00,  9.24it/s]


                   all        766        387      0.622      0.433      0.479      0.211
               Handgun        163        254      0.647      0.469      0.517      0.229
           Short_rifle         82        100      0.692       0.66      0.706      0.316
                 Knife         33         33      0.528       0.17      0.215     0.0879
Speed: 0.1ms preprocess, 1.3ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x786778109090>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

YOLOV11s

In [ ]:
!yolo detect train data=/content/data.yaml model=yolo11s.pt epochs=30 imgsz=640
!cp -r /runs/train/exp /content/drive/MyDrive/RESULTS_FINAL/

100% 18.4M/18.4M [00:00<00:00, 436MB/s]
Ultralytics 8.3.127 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (NVIDIA L4, 22693MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/data.yaml, degrees=0.0, deterministic=True, device=cuda:0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0,

In [ ]:
!ls -R runs/detect/train2

runs/detect/train2:
args.yaml			 R_curve.png	      train_batch4482.jpg
confusion_matrix_normalized.png  results.csv	      val_batch0_labels.jpg
confusion_matrix.png		 results.png	      val_batch0_pred.jpg
F1_curve.png			 train_batch0.jpg     val_batch1_labels.jpg
labels_correlogram.jpg		 train_batch1.jpg     val_batch1_pred.jpg
labels.jpg			 train_batch2.jpg     val_batch2_labels.jpg
P_curve.png			 train_batch4480.jpg  val_batch2_pred.jpg
PR_curve.png			 train_batch4481.jpg  weights

runs/detect/train2/weights:
best.pt  last.pt


**Download results**

In [ ]:
# Zip the folder
!zip -r exp.zip runs/train/exp2
# Download to your local machine
from google.colab import files
files.download("exp.zip")

  adding: runs/train/exp2/ (stored 0%)
  adding: runs/train/exp2/labels_correlogram.jpg (deflated 37%)
  adding: runs/train/exp2/P_curve.png (deflated 11%)
  adding: runs/train/exp2/labels.jpg (deflated 33%)
  adding: runs/train/exp2/val_batch0_pred.jpg (deflated 13%)
  adding: runs/train/exp2/val_batch2_labels.jpg (deflated 9%)
  adding: runs/train/exp2/events.out.tfevents.1746511661.7c679906dde0.2337.0 (deflated 31%)
  adding: runs/train/exp2/val_batch2_pred.jpg (deflated 9%)
  adding: runs/train/exp2/opt.yaml (deflated 49%)
  adding: runs/train/exp2/confusion_matrix.png (deflated 30%)
  adding: runs/train/exp2/weights/ (stored 0%)
  adding: runs/train/exp2/weights/last.pt (deflated 9%)
  adding: runs/train/exp2/weights/best.pt (deflated 9%)
  adding: runs/train/exp2/train_batch1.jpg (deflated 6%)
  adding: runs/train/exp2/train_batch2.jpg (deflated 5%)
  adding: runs/train/exp2/F1_curve.png (deflated 10%)
  adding: runs/train/exp2/train_batch0.jpg (deflated 7%)
  adding: runs/train/

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>